# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [15]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [16]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,20.78,78,99,10.64,PN,1730321692
1,1,douentza,14.9951,-2.9517,29.06,17,83,1.38,ML,1730321693
2,2,yerofey pavlovich,53.9667,121.9667,-4.05,96,100,1.14,RU,1730321694
3,3,kihei,20.7850,-156.4656,27.81,79,40,9.77,US,1730321695
4,4,gizo,-8.1030,156.8419,27.79,76,57,0.83,SB,1730321696


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [17]:
%%capture --no-display

# Configure the map plot
each_city = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

# Display the map
each_city

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [18]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[
    (city_data_df['Max Temp'] >= 20) &
    (city_data_df['Max Temp'] <= 30) &
    (city_data_df['Wind Speed'] < 10) &
    (city_data_df['Cloudiness'] < 50)
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,kihei,20.7850,-156.4656,27.81,79,40,9.77,US,1730321695
12,12,kourou,5.1552,-52.6478,27.57,76,38,3.40,GF,1730321705
20,20,lazaro cardenas,17.9583,-102.2000,29.51,70,2,3.57,MX,1730321638
22,22,gebeit,21.0667,36.3167,25.43,66,0,2.35,SD,1730321717
27,27,port blair,11.6667,92.7500,26.41,83,12,1.67,IN,1730321723


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,adamstown,PN,-25.0660,-130.1015,78,
1,douentza,ML,14.9951,-2.9517,17,
2,yerofey pavlovich,RU,53.9667,121.9667,96,
3,kihei,US,20.7850,-156.4656,79,
4,gizo,SB,-8.1030,156.8419,76,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()


    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
adamstown - nearest hotel: No hotel found
douentza - nearest hotel: No hotel found
yerofey pavlovich - nearest hotel: No hotel found
kihei - nearest hotel: Maui Beach Resort
gizo - nearest hotel: Gizo Hotel
ovalle - nearest hotel: Hotel Roxy
lihue - nearest hotel: Kauai Palms
port-aux-francais - nearest hotel: Keravel
olonkinbyen - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
invercargill - nearest hotel: The Grand
paramonga - nearest hotel: Hotel Rico
kourou - nearest hotel: SCI Horizon
lucapa - nearest hotel: No hotel found
muhlhausen - nearest hotel: Ammerscher Bahnhof
isafjordur - nearest hotel: Hótel Horn
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
kudymkar - nearest hotel: No hotel found
la ronge - nearest hotel: La Ronge Hotel and Suites
bredasdorp - nearest hotel: Victoria Hotel
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
margaret river - nearest hotel: Margaret River Hotel
gebeit - nearest hotel: No hotel f

,City,Country,Lat,Lng,Humidity,Hotel Name
0,adamstown,PN,-25.0660,-130.1015,78,No hotel found
1,douentza,ML,14.9951,-2.9517,17,No hotel found
2,yerofey pavlovich,RU,53.9667,121.9667,96,No hotel found
3,kihei,US,20.7850,-156.4656,79,Maui Beach Resort
4,gizo,SB,-8.1030,156.8419,76,Gizo Hotel
...,...,...,...,...,...,...
564,ongjin,KP,37.9347,125.3619,65,No hotel found
565,pierre,US,44.3683,-100.3510,54,Quality Inn
566,anton lizardo,MX,19.0500,-95.9833,75,Hotel CocoAventura
567,mu'minobod,TJ,38.1085,70.0285,48,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    alpha =0.8,
    color = 'City',
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)